# Tensorflow Object Detection API and AWS Sagemaker

In this notebook, you will train and evaluate different models using the [Tensorflow Object Detection API](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/) and [AWS Sagemaker](https://aws.amazon.com/sagemaker/). 

If you ever feel stuck, you can refer to this [tutorial](https://aws.amazon.com/blogs/machine-learning/training-and-deploying-models-using-tensorflow-2-with-the-object-detection-api-on-amazon-sagemaker/).

## Dataset

We are using the [Waymo Open Dataset](https://waymo.com/open/) for this project. The dataset has already been exported using the tfrecords format. The files have been created following the format described [here](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html#create-tensorflow-records). You can find data stored on [AWS S3](https://aws.amazon.com/s3/), AWS Object Storage. The images are saved with a resolution of 640x640.

In [1]:
%%capture
%pip install tensorflow_io sagemaker -U

In [2]:
import os
import sagemaker
from sagemaker.estimator import Estimator
from framework import CustomFramework

Save the IAM role in a variable called `role`. This would be useful when training the model.

In [3]:
role = sagemaker.get_execution_role()
print(role)

arn:aws:iam::382683543627:role/service-role/AmazonSageMaker-ExecutionRole-20230304T140215


In [4]:
# The train and val paths below are public S3 buckets created by Udacity for this project
inputs = {'train': 's3://cd2688-object-detection-tf2/train/', 
        'val': 's3://cd2688-object-detection-tf2/val/'} 

# Insert path of a folder in your personal S3 bucket to store tensorboard logs.
tensorboard_s3_prefix = 's3://object-detection-project-1/logs/'

## Container

To train the model, you will first need to build a [docker](https://www.docker.com/) container with all the dependencies required by the TF Object Detection API. The code below does the following:
* clone the Tensorflow models repository
* get the exporter and training scripts from the the repository
* build the docker image and push it 
* print the container name

In [5]:
%%bash

# clone the repo and get the scripts
git clone https://github.com/tensorflow/models.git docker/models

# get model_main and exporter_main files from TF2 Object Detection GitHub repository
cp docker/models/research/object_detection/exporter_main_v2.py source_dir 
cp docker/models/research/object_detection/model_main_tf2.py source_dir

Cloning into 'docker/models'...


In [6]:
# build and push the docker image. This code can be commented after being ran once.
# This will take around 10 mins.
image_name = 'tf2-object-detection'
!sh ./docker/build_and_push.sh $image_name

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Building image with name tf2-object-detection
Sending build context to Docker daemon  723.1MB
Step 1/17 : FROM tensorflow/tensorflow:2.9.0-gpu
2.9.0-gpu: Pulling from tensorflow/tensorflow

17ec1767: Pulling fs layer 
9ecd2bff: Pulling fs layer 
4ae53552: Pulling fs layer 
2d09b8c4: Pulling fs layer 
0d530989: Pulling fs layer 
81af025b: Pulling fs layer 
c129f45e: Pulling fs layer 
8fcb70c6: Pulling fs layer 
9aa4a247: Pulling fs layer 
3100c8d1: Pulling fs layer 
3a6b487b: Pulling fs layer 
e8773234: Pulling fs layer 
36c9476c: Pulling fs layer 


e8773234: Extracting  184.9MB/583.3MBBDownloading  85.53kB/7.929MBDownloading  590.2MB/1.913GB

Digest: sha256:aa9f4a6a7debc976135702118aedfd0d72bf9e495af6ecfd5a31d9714e3354263A
Status: Downloaded newer image for tensorflow/tensorflow:2.9.0-gpu
 ---> c8d9ee2a0ff4
Step 2/17 : ARG DEBIAN_FRONTEND=noninteractive
 ---> Running in c9abf4708e05
Removing intermediate container c9abf4708e05
 ---> 94cafd45bd4f
Step 3/17 : RUN rm /etc/apt/sources.list.d/cuda.list
 ---> Running in a339cd645007
Removing intermediate container a339cd645007
 ---> 3a1812d59ed1
Step 4/17 : RUN apt-key del 7fa2af80
 ---> Running in d02b9fb4b875
OK
Removing intermediate container d02b9fb4b875
 ---> dc048943e9cc
Step 5/17 : RUN apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/3bf863cc.pub
 ---> Running in b0b7c2a1e298
Executing: /tmp/apt-key-gpghome.5vsrymR99P/gpg.1.sh --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/3bf863cc.pub
gpg: requesting key from 'https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_

11 upgraded, 119 newly installed, 0 to remove and 109 not upgraded.
Need to get 59.5 MB of archives.
After this operation, 631 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 gpg-wks-client amd64 2.2.19-3ubuntu2.2 [97.4 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 dirmngr amd64 2.2.19-3ubuntu2.2 [330 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 gpg-wks-server amd64 2.2.19-3ubuntu2.2 [90.2 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 gnupg-utils amd64 2.2.19-3ubuntu2.2 [481 kB]
Get:5 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 gpg-agent amd64 2.2.19-3ubuntu2.2 [232 kB]
Get:6 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 gpg amd64 2.2.19-3ubuntu2.2 [482 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 gpgconf amd64 2.2.19-3ubuntu2.2 [124 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 gnupg-l10n all

Get:79 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libglx0 amd64 1.3.2-1~ubuntu0.20.04.2 [32.5 kB]
Get:80 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libgl1 amd64 1.3.2-1~ubuntu0.20.04.2 [85.8 kB]
Get:81 http://archive.ubuntu.com/ubuntu focal/main amd64 xorg-sgml-doctools all 1:1.11-1 [12.9 kB]
Get:82 http://archive.ubuntu.com/ubuntu focal/main amd64 x11proto-dev all 2019.2-1ubuntu1 [594 kB]
Get:83 http://archive.ubuntu.com/ubuntu focal/main amd64 x11proto-core-dev all 2019.2-1ubuntu1 [2620 B]
Get:84 http://archive.ubuntu.com/ubuntu focal/main amd64 libxau-dev amd64 1:1.0.9-0ubuntu1 [9552 B]
Get:85 http://archive.ubuntu.com/ubuntu focal/main amd64 libxdmcp-dev amd64 1:1.1.3-0ubuntu1 [25.3 kB]
Get:86 http://archive.ubuntu.com/ubuntu focal/main amd64 xtrans-dev all 1.4.0-1 [68.9 kB]
Get:87 http://archive.ubuntu.com/ubuntu focal/main amd64 libpthread-stubs0-dev amd64 0.4-1 [5384 B]
Get:88 http://archive.ubuntu.com/ubuntu focal/main amd64 libxcb1-dev amd64 1.14-

...

7118a62: Pushing  1.033GB/3.353GBPushing  549.4MB/3.353GBPushing  495.4MB/3.681GB

574c12e: Pushing  1.367GB/1.455GB

fae0f58c: Pushing  2.417GB/3.681GBPushing  1.625GB/3.353GB

7118a62: Pushing  3.197GB/3.353GBPushing  2.926GB/3.681GB

7118a62: Pushed   3.385GB/3.353GB20230304083926: digest: sha256:4b9465860427acd2529c0425030afa03de2ab87891511b60b1d928ce99665e85 size: 5782
Saving ECR image URI into ecr_image_fullname.txt


To verify that the image was correctly pushed to the [Elastic Container Registry](https://aws.amazon.com/ecr/), you can look at it in the AWS webapp. For example, below you can see that three different images have been pushed to ECR. You should only see one, called `tf2-object-detection`.
![ECR Example](../data/example_ecr.png)


In [7]:
# display the container name
with open (os.path.join('docker', 'ecr_image_fullname.txt'), 'r') as f:
    container = f.readlines()[0][:-1]

print(container)

382683543627.dkr.ecr.us-east-1.amazonaws.com/tf2-object-detection:20230304083926


## Pre-trained model from model zoo

As often, we are not training from scratch and we will be using a pretrained model from the TF Object Detection model zoo. You can find pretrained checkpoints [here](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md). Because your time is limited for this project, we recommend to only experiment with the following models:
* SSD MobileNet V2 FPNLite 640x640	
* SSD ResNet50 V1 FPN 640x640 (RetinaNet50)	
* Faster R-CNN ResNet50 V1 640x640	
* EfficientDet D1 640x640	
* Faster R-CNN ResNet152 V1 640x640	

In the code below, the EfficientDet D1 model is downloaded and extracted. This code should be ajusted if you were to experiment with other architectures.

In [12]:
%%bash
mkdir /tmp/checkpoint
mkdir source_dir/checkpoint
wget -O /tmp/efficientdet.tar.gz http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet101_v1_640x640_coco17_tpu-8.tar.gz
tar -zxvf /tmp/efficientdet.tar.gz --strip-components 2 --directory source_dir/checkpoint faster_rcnn_resnet101_v1_640x640_coco17_tpu-8/checkpoint

mkdir: cannot create directory ‘/tmp/checkpoint’: File exists
mkdir: cannot create directory ‘source_dir/checkpoint’: File exists
--2023-03-04 09:25:48--  http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet101_v1_640x640_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 172.253.115.128, 2607:f8b0:4004:c06::80
Connecting to download.tensorflow.org (download.tensorflow.org)|172.253.115.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 353643040 (337M) [application/x-tar]
Saving to: ‘/tmp/efficientdet.tar.gz’

     0K .......... .......... .......... .......... ..........  0% 5.11M 66s
    50K .......... .......... .......... .......... ..........  0% 24.4M 40s
   100K .......... .......... .......... .......... ..........  0% 38.2M 30s
   150K .......... .......... .......... .......... ..........  0% 21.0M 26s
   200K .......... .......... .......... .......... ..........  0% 41.0M 23s
   250K

  5000K .......... .......... .......... .......... ..........  1%  308M 10s
  5050K .......... .......... .......... .......... ..........  1%  337M 10s
  5100K .......... .......... .......... .......... ..........  1%  341M 10s
  5150K .......... .......... .......... .......... ..........  1%  216M 10s
  5200K .......... .......... .......... .......... ..........  1%  297M 10s
  5250K .......... .......... .......... .......... ..........  1%  325M 10s
  5300K .......... .......... .......... .......... ..........  1%  348M 10s
  5350K .......... .......... .......... .......... ..........  1%  339M 10s
  5400K .......... .......... .......... .......... ..........  1% 20.4M 10s
  5450K .......... .......... .......... .......... ..........  1%  333M 9s
  5500K .......... .......... .......... .......... ..........  1%  365M 9s
  5550K .......... .......... .......... .......... ..........  1% 43.3M 9s
  5600K .......... .......... .......... .......... ..........  1%  311M 9s
  5

 10400K .......... .......... .......... .......... ..........  3%  318M 9s
 10450K .......... .......... .......... .......... ..........  3%  319M 9s
 10500K .......... .......... .......... .......... ..........  3%  325M 9s
 10550K .......... .......... .......... .......... ..........  3%  371M 8s
 10600K .......... .......... .......... .......... ..........  3%  288M 8s
 10650K .......... .......... .......... .......... ..........  3%  325M 8s
 10700K .......... .......... .......... .......... ..........  3% 18.6M 8s
 10750K .......... .......... .......... .......... ..........  3% 35.8M 8s
 10800K .......... .......... .......... .......... ..........  3%  293M 8s
 10850K .......... .......... .......... .......... ..........  3%  195M 8s
 10900K .......... .......... .......... .......... ..........  3%  107M 8s
 10950K .......... .......... .......... .......... ..........  3%  352M 8s
 11000K .......... .......... .......... .......... ..........  3%  315M 8s
 11050K ....

 15800K .......... .......... .......... .......... ..........  4%  272M 7s
 15850K .......... .......... .......... .......... ..........  4%  362M 7s
 15900K .......... .......... .......... .......... ..........  4%  360M 7s
 15950K .......... .......... .......... .......... ..........  4%  339M 7s
 16000K .......... .......... .......... .......... ..........  4%  318M 7s
 16050K .......... .......... .......... .......... ..........  4%  370M 7s
 16100K .......... .......... .......... .......... ..........  4%  304M 6s
 16150K .......... .......... .......... .......... ..........  4%  366M 6s
 16200K .......... .......... .......... .......... ..........  4%  279M 6s
 16250K .......... .......... .......... .......... ..........  4%  323M 6s
 16300K .......... .......... .......... .......... ..........  4%  313M 6s
 16350K .......... .......... .......... .......... ..........  4% 17.0M 6s
 16400K .......... .......... .......... .......... ..........  4%  130M 6s
 16450K ....

 21200K .......... .......... .......... .......... ..........  6%  324M 6s
 21250K .......... .......... .......... .......... ..........  6%  327M 6s
 21300K .......... .......... .......... .......... ..........  6%  312M 6s
 21350K .......... .......... .......... .......... ..........  6%  362M 6s
 21400K .......... .......... .......... .......... ..........  6% 9.46M 6s
 21450K .......... .......... .......... .......... ..........  6%  347M 6s
 21500K .......... .......... .......... .......... ..........  6%  324M 6s
 21550K .......... .......... .......... .......... ..........  6%  371M 6s
 21600K .......... .......... .......... .......... ..........  6%  324M 6s
 21650K .......... .......... .......... .......... ..........  6%  335M 6s
 21700K .......... .......... .......... .......... ..........  6%  373M 5s
 21750K .......... .......... .......... .......... ..........  6%  362M 5s
 21800K .......... .......... .......... .......... ..........  6%  319M 5s
 21850K ....

...

280400K .......... .......... .......... .......... .......... 81%  257M 1s
280450K .......... .......... .......... .......... .......... 81%  291M 1s
280500K .......... .......... .......... .......... .......... 81%  317M 1s
280550K .......... .......... .......... .......... .......... 81%  263M 1s
280600K .......... .......... .......... .......... .......... 81%  225M 1s
280650K .......... .......... .......... .......... .......... 81%  306M 1s
280700K .......... .......... .......... .......... .......... 81%  310M 1s
280750K .......... .......... .......... .......... .......... 81%  293M 1s
280800K .......... .......... .......... .......... .......... 81%  240M 1s
280850K .......... .......... .......... .......... .......... 81%  328M 1s
280900K .......... .......... .......... .......... .......... 81%  302M 1s
280950K .......... .......... .......... .......... .......... 81%  284M 1s
281000K .......... .......... .......... .......... .......... 81%  271M 1s
281050K ....

285800K .......... .......... .......... .......... .......... 82%  308M 1s
285850K .......... .......... .......... .......... .......... 82%  319M 1s
285900K .......... .......... .......... .......... .......... 82%  328M 1s
285950K .......... .......... .......... .......... .......... 82%  312M 1s
286000K .......... .......... .......... .......... .......... 82%  270M 1s
286050K .......... .......... .......... .......... .......... 82%  274M 1s
286100K .......... .......... .......... .......... .......... 82%  249M 1s
286150K .......... .......... .......... .......... .......... 82%  297M 1s
286200K .......... .......... .......... .......... .......... 82%  266M 1s
286250K .......... .......... .......... .......... .......... 82%  277M 1s
286300K .......... .......... .......... .......... .......... 82%  317M 1s
286350K .......... .......... .......... .......... .......... 82%  311M 1s
286400K .......... .......... .......... .......... .......... 82%  276M 1s
286450K ....

291200K .......... .......... .......... .......... .......... 84%  258M 1s
291250K .......... .......... .......... .......... .......... 84%  262M 1s
291300K .......... .......... .......... .......... .......... 84%  292M 1s
291350K .......... .......... .......... .......... .......... 84%  288M 1s
291400K .......... .......... .......... .......... .......... 84%  252M 1s
291450K .......... .......... .......... .......... .......... 84%  325M 1s
291500K .......... .......... .......... .......... .......... 84%  268M 1s
291550K .......... .......... .......... .......... .......... 84%  282M 1s
291600K .......... .......... .......... .......... .......... 84%  226M 1s
291650K .......... .......... .......... .......... .......... 84%  258M 1s
291700K .......... .......... .......... .......... .......... 84% 8.85M 1s
291750K .......... .......... .......... .......... .......... 84%  296M 1s
291800K .......... .......... .......... .......... .......... 84%  283M 1s
291850K ....

296600K .......... .......... .......... .......... .......... 85%  236M 1s
296650K .......... .......... .......... .......... .......... 85%  326M 1s
296700K .......... .......... .......... .......... .......... 85%  330M 1s
296750K .......... .......... .......... .......... .......... 85%  308M 1s
296800K .......... .......... .......... .......... .......... 85%  274M 1s
296850K .......... .......... .......... .......... .......... 85%  332M 1s
296900K .......... .......... .......... .......... .......... 85%  208M 1s
296950K .......... .......... .......... .......... .......... 85%  108M 1s
297000K .......... .......... .......... .......... .......... 86%  222M 1s
297050K .......... .......... .......... .......... .......... 86%  317M 1s
297100K .......... .......... .......... .......... .......... 86%  157M 1s
297150K .......... .......... .......... .......... .......... 86%  131M 1s
297200K .......... .......... .......... .......... .......... 86%  242M 1s
297250K ....

faster_rcnn_resnet101_v1_640x640_coco17_tpu-8/checkpoint/checkpoint
faster_rcnn_resnet101_v1_640x640_coco17_tpu-8/checkpoint/ckpt-0.index


## Edit pipeline.config file

The [`pipeline_ssdresnet.config`](source_dir/pipeline.config) in the `source_dir` folder should be updated when you experiment with different models. The different config files are available [here](https://github.com/tensorflow/models/tree/master/research/object_detection/configs/tf2).

>Note: The provided `pipeline.config` file works well with the `EfficientDet` model. You would need to modify it when working with other models.

## Launch Training Job

Now that we have a dataset, a docker image and some pretrained model weights, we can launch the training job. To do so, we create a [Sagemaker Framework](https://sagemaker.readthedocs.io/en/stable/frameworks/index.html), where we indicate the container name, name of the config file, number of training steps etc.

The `run_training.sh` script does the following:
* train the model for `num_train_steps` 
* evaluate over the val dataset
* export the model

Different metrics will be displayed during the evaluation phase, including the mean average precision. These metrics can be used to quantify your model performances and compare over the different iterations.

You can also monitor the training progress by navigating to **Training -> Training Jobs** from the Amazon Sagemaker dashboard in the Web UI.

In [13]:
tensorboard_output_config = sagemaker.debugger.TensorBoardOutputConfig(
    s3_output_path=tensorboard_s3_prefix,
    container_local_output_path='/opt/training/'
)

estimator = CustomFramework(
    role=role,
    image_uri=container,
    entry_point='run_training.sh',
    source_dir='source_dir/',
    hyperparameters={
        "model_dir":"/opt/training",        
        "pipeline_config_path": "pipeline.config",
        "num_train_steps": "2000",    
        "sample_1_of_n_eval_examples": "1"
    },
    instance_count=1,
    instance_type='ml.p3.2xlarge',
    tensorboard_output_config=tensorboard_output_config,
    disable_profiler=True,
    base_job_name='tf2-object-detection'
)

estimator.fit(inputs)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:sagemaker:Creating training-job with name: tf2-object-detection-2023-03-04-09-26-15-201


2023-03-04 09:26:28 Starting - Starting the training job...
2023-03-04 09:26:56 Starting - Preparing the instances for training.........
2023-03-04 09:28:24 Downloading - Downloading input data
2023-03-04 09:28:24 Training - Downloading the training image...............
2023-03-04 09:30:50 Training - Training image download completed. Training in progress....2023-03-04 09:31:20,466 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-03-04 09:31:20,500 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-03-04 09:31:20,533 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-03-04 09:31:20,548 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "train": "/opt/ml/input/data/train",
        "val": "/opt/ml/input/data/val"
    },
    "current_host": "algo-1",
    "curre

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I0304 09:31:28.390519 140664591980352 mirrored_strategy.py:374] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: 2000
I0304 09:31:28.394237 140664591980352 config_util.py:552] Maybe overwriting train_steps: 2000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0304 09:31:28.394369 140664591980352 config_util.py:552] Maybe overwriting use_bfloat16: False
Instructions for updating:
rename to distribute_datasets_from_function
W0304 09:31:28.435765 140664591980352 deprecation.py:350] From /usr/local/lib/python3.8/dist-packages/object_detection/model_lib_v2.py:563: StrategyBase.experimental_distribute_datasets_from_function (from tensorflow.python.distribute.distribute_lib) is deprecated and will be removed in a future version.
Instructions for updating:
rename to distribute_datasets_from_function
INFO

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
I0304 09:32:25.034352 140664591980352 cross_device_ops.py:616] Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
I0304 09:32:25.037964 140664591980352 cross_device_ops.py:616] Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
I0304 09:32:25.039150 140664591980352 cross_device_ops.py:616] Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Red

INFO:tensorflow:Step 600 per-step time 0.195s
I0304 09:35:30.208870 140664591980352 model_lib_v2.py:705] Step 600 per-step time 0.195s
INFO:tensorflow:{'Loss/BoxClassifierLoss/classification_loss': 0.17778042,
 'Loss/BoxClassifierLoss/localization_loss': 0.33501524,
 'Loss/RPNLoss/localization_loss': 0.17367558,
 'Loss/RPNLoss/objectness_loss': 0.13796325,
 'Loss/regularization_loss': 0.0,
 'Loss/total_loss': 0.8244345,
 'learning_rate': 0.01036}
I0304 09:35:30.209228 140664591980352 model_lib_v2.py:708] {'Loss/BoxClassifierLoss/classification_loss': 0.17778042,
 'Loss/BoxClassifierLoss/localization_loss': 0.33501524,
 'Loss/RPNLoss/localization_loss': 0.17367558,
 'Loss/RPNLoss/objectness_loss': 0.13796325,
 'Loss/regularization_loss': 0.0,
 'Loss/total_loss': 0.8244345,
 'learning_rate': 0.01036}
INFO:tensorflow:Step 700 per-step time 0.196s
I0304 09:35:49.806291 140664591980352 model_lib_v2.py:705] Step 700 per-step time 0.196s
INFO:tensorflow:{'Loss/BoxClassifierLoss/classification

INFO:tensorflow:Step 1600 per-step time 0.197s
I0304 09:38:48.776377 140664591980352 model_lib_v2.py:705] Step 1600 per-step time 0.197s
INFO:tensorflow:{'Loss/BoxClassifierLoss/classification_loss': 0.21161649,
 'Loss/BoxClassifierLoss/localization_loss': 0.5284993,
 'Loss/RPNLoss/localization_loss': 0.7393585,
 'Loss/RPNLoss/objectness_loss': 0.1030938,
 'Loss/regularization_loss': 0.0,
 'Loss/total_loss': 1.582568,
 'learning_rate': 0.025959998}
I0304 09:38:48.776759 140664591980352 model_lib_v2.py:708] {'Loss/BoxClassifierLoss/classification_loss': 0.21161649,
 'Loss/BoxClassifierLoss/localization_loss': 0.5284993,
 'Loss/RPNLoss/localization_loss': 0.7393585,
 'Loss/RPNLoss/objectness_loss': 0.1030938,
 'Loss/regularization_loss': 0.0,
 'Loss/total_loss': 1.582568,
 'learning_rate': 0.025959998}
INFO:tensorflow:Step 1700 per-step time 0.199s
I0304 09:39:08.713376 140664591980352 model_lib_v2.py:705] Step 1700 per-step time 0.199s
INFO:tensorflow:{'Loss/BoxClassifierLoss/classifica

/usr/local/lib/python3.8/dist-packages/keras/backend.py:451: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(
INFO:tensorflow:depth of additional conv before box predictor: 0
I0304 09:40:35.976139 139940740056896 convolutional_keras_box_predictor.py:152] depth of additional conv before box predictor: 0
Instructions for updating:
Use ref() instead.
W0304 09:40:43.102751 139940740056896 deprecation.py:350] From /usr/local/lib/python3.8/dist-packages/tensorflow/python/autograph/impl/api.py:459: Tensor.experimental_ref (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use ref() instead.
Instructions for updating:
Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.
See `tf.n

INFO:tensorflow:Waiting for new checkpoint at /opt/training
I0304 09:45:26.401811 139940740056896 checkpoint_utils.py:140] Waiting for new checkpoint at /opt/training
INFO:tensorflow:Timed-out waiting for a checkpoint.
I0304 09:45:35.417552 139940740056896 checkpoint_utils.py:203] Timed-out waiting for a checkpoint.
creating index...
index created!
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=20.16s).
Accumulating evaluation results...
DONE (t=0.51s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.017
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.049
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.007
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.005
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.061
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.124
 Ave

In [16]:
job_artifacts_path = estimator.latest_job_tensorboard_artifacts_path()
tensorboard_s3_output_path = f'{job_artifacts_path}/train'

!F_CPP_MIN_LOG_LEVEL=3 AWS_REGION='us-east-1' tensorboard --logdir=$tensorboard_s3_output_path

TensorFlow installation not found - running with reduced feature set.
Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.12.0 at http://localhost:6006/ (Press CTRL+C to quit)
^C


In [15]:
pip install tensorboard

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 67.8 MB/s eta 0:00:00:00:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 92.3 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.3/93.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 33.3 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/177.2 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 43.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should be able to see your model training in the AWS webapp as shown below:
![ECR Example](../data/example_trainings.png)


## Improve on the intial model

Most likely, this initial experiment did not yield optimal results. However, you can make multiple changes to the `pipeline.config` file to improve this model. One obvious change consists in improving the data augmentation strategy. The [`preprocessor.proto`](https://github.com/tensorflow/models/blob/master/research/object_detection/protos/preprocessor.proto) file contains the different data augmentation method available in the Tf Object Detection API. Justify your choices of augmentations in the writeup.

Keep in mind that the following are also available:
* experiment with the optimizer: type of optimizer, learning rate, scheduler etc
* experiment with the architecture. The Tf Object Detection API model zoo offers many architectures. Keep in mind that the pipeline.config file is unique for each architecture and you will have to edit it.
* visualize results on the test frames using the `2_deploy_model` notebook available in this repository.

In the cell below, write down all the different approaches you have experimented with, why you have chosen them and what you would have done if you had more time and resources. Justify your choices using the tensorboard visualizations (take screenshots and insert them in your writeup), the metrics on the evaluation set and the generated animation you have created with [this tool](../2_run_inference/2_deploy_model.ipynb).

## Write-up - it lies in Faster RCNN notebook
